In [70]:
import pandas as pd
import streamlit as st
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
import pickle
import seaborn as sns


In [71]:
data = pd.read_csv('Expresso_churn_dataset.csv')
data.head()

,REGION,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,MRG,REGULARITY,TOP_PACK,FREQ_TOP_PACK,CHURN
0,FATICK,K > 24 month,4250.0,15.0,4251.0,1417.0,17.0,4.0,388.0,46.0,NO,54,On net 200F=Unlimited _call24H,8.0,0
1,NaN,I 18-21 month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,4,NaN,NaN,1
2,NaN,K > 24 month,3600.0,2.0,1020.0,340.0,2.0,NaN,90.0,46.0,NO,17,On-net 1000F=10MilF;10d,1.0,0
3,DAKAR,K > 24 month,13500.0,15.0,13502.0,4501.0,18.0,43804.0,41.0,102.0,NO,62,"Data:1000F=5GB,7d",11.0,0
4,DAKAR,K > 24 month,1000.0,1.0,985.0,328.0,1.0,NaN,39.0,24.0,NO,11,Mixt 250F=Unlimited_call24H,2.0,0


In [72]:
df = data.copy()

In [73]:
# df.drop('user_id', axis = 1, inplace = True)
# for i in df:
#     if df[i].isnull().sum() / len(df) * 100 > 50:
#         df.drop(i, axis = 1, inplace = True)
# df.isnull().sum()

In [74]:
# sampling0 = df[df.CHURN == 0]
# sampling0 = sampling0.dropna()
# sampling0.reset_index(drop = True, inplace = True)
# sampling0.shape

In [75]:
# sampling1 = df[df.CHURN == 1]
# sampling1.drop(['REGION', 'TOP_PACK', 'FREQ_TOP_PACK'], axis = 1, inplace = True)
# sampling1.dropna(inplace = True)
# sampling1.reset_index(drop = True, inplace =True)
# sampling1.shape

In [76]:
# sampling0 = sampling0.sample(35000)

# cols = sampling1.columns

# dx = pd.concat([sampling1, sampling0[cols]], axis = 0)
# dx.isnull().sum()


In [77]:
# sns.countplot(x = dx.CHURN)
# dx.CHURN.value_counts()

In [78]:
# dx.dropna()

In [79]:
# Check if the DEPENDENTS column has been identified into its right datatype
categoricals = df.select_dtypes(include = ['object', 'category'])
numericals = df.select_dtypes(include = 'number')

print(f"\t\tCategorical Columns")
display(categoricals.head())
print(f"\n\t\tNumerical Columns")
display(numericals.head())

		Categorical Columns


,REGION,TENURE,MRG,TOP_PACK
0,FATICK,K > 24 month,NO,On net 200F=Unlimited _call24H
1,NaN,I 18-21 month,NO,NaN
2,NaN,K > 24 month,NO,On-net 1000F=10MilF;10d
3,DAKAR,K > 24 month,NO,"Data:1000F=5GB,7d"
4,DAKAR,K > 24 month,NO,Mixt 250F=Unlimited_call24H



		Numerical Columns


,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,REGULARITY,FREQ_TOP_PACK,CHURN
0,4250.0,15.0,4251.0,1417.0,17.0,4.0,388.0,46.0,54,8.0,0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,1
2,3600.0,2.0,1020.0,340.0,2.0,NaN,90.0,46.0,17,1.0,0
3,13500.0,15.0,13502.0,4501.0,18.0,43804.0,41.0,102.0,62,11.0,0
4,1000.0,1.0,985.0,328.0,1.0,NaN,39.0,24.0,11,2.0,0


In [80]:
def outlierRemoval(dataframe):
    for i in dataframe.columns:
        if dataframe[i].dtypes != 'O': # --------------------------------------- If the data type is not an object type
            Q1 = dataframe[i].describe()[4] # ---------------------------------- Identify lower Quartile
            Q3 = dataframe[i].describe()[6] # ---------------------------------- Identify the upper quartile
            IQR = Q3 - Q1 # ---------------------------------------------------- Get Inter Quartile Range
            minThreshold = Q1 - 1.5 * IQR # ------------------------------------ Get Minimum Threshold
            maxThreshold = Q3 + 1.5 * IQR # ------------------------------------ Get Maximum Threshold

            dataframe = dataframe.loc[(dataframe[i] >= minThreshold) & (dataframe[i] <= maxThreshold)]
    return dataframe


df = outlierRemoval(df)

In [81]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
scaler = StandardScaler()
encoder = LabelEncoder()

for i in numericals.columns: # ................................................. Select all numerical columns
    if i in df.drop('CHURN', axis = 1).columns: # ...................................................... If the selected column is found in the general dataframe
        df[i] = scaler.fit_transform(df[[i]]) # ................................ Scale it

for i in categoricals.columns: # ............................................... Select all categorical columns
    if i in df.drop('CHURN', axis = 1).columns: # ...................................................... If the selected columns are found in the general dataframe
        df[i] = encoder.fit_transform(df[i])# .................................. encode it

df.head()

sel_cols = ['REGULARITY', 'DATA_VOLUME','REVENUE',  'ORANGE', 'ON_NET', 'MONTANT','FREQUENCE']
x = df[sel_cols]

x = x
y = df.CHURN
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size = 0.20, random_state = 75, stratify = y)

In [82]:
# Modelling
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
model = RandomForestClassifier() 
model.fit(xtrain, ytrain) 
cross_validation = model.predict(xtrain)
pred = model.predict(xtest)

In [83]:
model = pickle.dump(model, open('Expresso_churn.pkl', 'wb'))

print(f'\nModel is Saved\n')


Model is Saved

